In [30]:
%pip install catboost

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [32]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [33]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [34]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [36]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import RandomizedSearchCV, train_test_split

k = 75  # Change this value to select a different number of top features
selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_testdata_selected = selector.transform(X_testdata)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

Selected features: Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X34', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44',
       'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54',
       'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64',
       'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74',
       'X75', 'X76', 'X77', 'X78'],
      dtype='object')


In [29]:
param_grid = {
    'iterations': [1000, 2000, 3000],
    'learning_rate': [0.01, 0.03, 0.1],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0.5, 1, 2],
    'border_count': [32, 64, 128],
    'random_strength': [1, 1.5, 2],
    'rsm': [0.8, 0.9, 1.0],  # Random subspace method
    'scale_pos_weight': [1, 2, 3]  # Balancing the positive class
}

# Initialize the CatBoost model
catboost_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    task_type='CPU',
    random_state=42
)

# Perform grid search
random_search = RandomizedSearchCV(estimator=catboost_model, param_distributions=param_grid, n_iter=6, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1, random_state=42)
random_search.fit(X_train_selected, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [38]:
# Print the best parameters and the best score
# print("Best parameters found: ", random_search.best_params_)
# print("Best AUC score: ", random_search.best_score_)
# param_grid = {
#     'iterations': [1000, 2000, 3000],
#     'learning_rate': [0.01, 0.03, 0.1],
#     'depth': [6, 8, 10],
#     'l2_leaf_reg': [1, 3, 5, 7],
#     'bagging_temperature': [0.5, 1, 2],
#     'border_count': [32, 64, 128],
#     'random_strength': [1, 1.5, 2],
#     'rsm': [0.8, 0.9, 1.0],  # Random subspace method
#     'scale_pos_weight': [1, 2, 3]  # Balancing the positive class
# }
# Train the model using the best parameters
# best_params = random_search.best_params_
best_catboost_model = CatBoostClassifier(
    iterations=15000,
    loss_function='Logloss',
    learning_rate=0.1,
    eval_metric='AUC',
    task_type='CPU',
    random_state=42,
    
)
best_catboost_model.fit(X_train_selected, y_train)

y_pred = best_catboost_model.predict(X_test_selected)
y_pred_proba = best_catboost_model.predict_proba(X_test_selected)[:, 1]

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_proba))

0:	total: 117ms	remaining: 29m 14s
1:	total: 216ms	remaining: 26m 56s
2:	total: 358ms	remaining: 29m 51s
3:	total: 484ms	remaining: 30m 14s
4:	total: 606ms	remaining: 30m 17s
5:	total: 724ms	remaining: 30m 8s
6:	total: 812ms	remaining: 28m 58s
7:	total: 910ms	remaining: 28m 25s
8:	total: 1.01s	remaining: 27m 58s
9:	total: 1.11s	remaining: 27m 50s
10:	total: 1.23s	remaining: 28m
11:	total: 1.38s	remaining: 28m 38s
12:	total: 1.48s	remaining: 28m 29s
13:	total: 1.59s	remaining: 28m 17s
14:	total: 1.7s	remaining: 28m 13s
15:	total: 1.77s	remaining: 27m 36s
16:	total: 1.84s	remaining: 27m 4s
17:	total: 1.94s	remaining: 26m 51s
18:	total: 2.03s	remaining: 26m 39s
19:	total: 2.13s	remaining: 26m 38s
20:	total: 2.22s	remaining: 26m 21s
21:	total: 2.29s	remaining: 25m 59s
22:	total: 2.36s	remaining: 25m 38s
23:	total: 2.43s	remaining: 25m 17s
24:	total: 2.51s	remaining: 25m 4s
25:	total: 2.59s	remaining: 24m 51s
26:	total: 2.66s	remaining: 24m 37s
27:	total: 2.74s	remaining: 24m 25s
28:	total: